In [204]:
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from camel_tools.tokenizers.word import simple_word_tokenize
import nltk
import pandas as pd
import plotly.express as px
import re
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [205]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohammedwalidadawy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [206]:
df = pd.read_csv('arabic_reviews.csv')
# Remove 'Mixed' class
df = df[df['label'] != 'Mixed']

In [207]:
df

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...


In [208]:
df[('label')].value_counts()

label
Positive    33333
Negative    33333
Name: count, dtype: int64

In [209]:
px.histogram(df, x="label") #check for imbalance

In [210]:
df.duplicated().sum() #check duplicates

0

In [211]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [212]:
print(df['label'].unique())

['Positive' 'Negative']


# PREPROCESSING

In [213]:
#stop_words = list(set(stopwords.words('arabic')))

stop_words = set([
    'في', 'من', 'إلى', 'على', 'أن', 'لا', 'ما', 'هذا', 'هذه', 'ذلك',
    'كان', 'يكون', 'هو', 'هي', 'مع', 'بين', 'عن', 'في', 'و', 'أو', 'إذ', 'إذا'
])

DIALECT_MSA_MAP = {
    # Egyptian
    "مش": "ليس",
    "دلوقتي": "الآن",
    "إزاي": "كيف",
    "كده": "هكذا",

    # Levantine
    "شو": "ماذا",
    "ليش": "لماذا",
    "كتير": "كثيراً",
    "مافي": "لا يوجد",

    # Gulf
    "وايد": "كثيراً",
    "زين": "جيد",
    "شسالفة": "ما القصة",
    "ايوا": "نعم",

    # Moroccan / Maghrebi
    "بزاف": "كثيراً",
    "واش": "هل",
    "شنو": "ماذا",
    "دابا": "الآن",

    # Iraqi
    "هسة": "الآن",
    "ماكو": "لا يوجد",
    "شكو ماكو": "ما الأخبار؟",
    "شلونك": "كيف حالك",
}

In [214]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [215]:
def dialect_to_msa(text):
    words = simple_word_tokenize(text)
    return ' '.join(DIALECT_MSA_MAP.get(word, word) for word in words)


In [216]:
def remove_gibberish(text):
    arabic_pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s.,!?؟]')
    return arabic_pattern.sub('', text)


In [217]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    return re.sub(arabic_diacritics, '', str(text))


In [218]:
def remove_emoji(text):
    regrex_pattern = re.compile("[\U0001F600-\U0001F64F"
                                "\U0001F300-\U0001F5FF"
                                "\U0001F680-\U0001F6FF"
                                "\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'', text)


In [219]:
def normalize_elongation(text):
    return re.sub(r'(.)\1+', r'\1\1', text)


In [220]:
def normalize_letters(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    return text


In [221]:
def clean_punctuation(text):
    # Define punctuation to keep
    keep = '،.؟!'  # Arabic comma, dot, question, exclamation

    # Remove any punctuation that is NOT in 'keep'
    # This keeps Arabic letters and spaces, removes other symbols
    text = re.sub(rf'[^\w\s{keep}]', '', text)

    #remove excessive dots like "..."
    text = re.sub(r'\.{2,}', '.', text)

    # remove multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [222]:
#pipline
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    # text = dialect_to_msa(text)
    text = remove_gibberish(text)
    text = remove_emoji(text)
    text = normalize_elongation(text)
    text = remove_diacritics(text)
    text = normalize_letters(text)
    text = clean_punctuation(text)
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text


In [223]:
df['cleanedtext'] = df['text'].apply(clean_text)

In [224]:
df

,label,text,cleanedtext
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,احد اسباب نجاح الامارات ان كل شخص الدولة يعشق ...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,هادفة وقوية تنقلك صخب شوارع القاهرة الي هدوء ج...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,خلصنا مبدءيا اللي مستني ابهار زي الفيل الازرق ...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,ياسات جلوريا جزء يتجزا دبي فندق متكامل الخدمات...
...,...,...,...
99994,Negative,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,معرفش ليه كنت عاوزة اكملها وهي مش عاجباني البد...
99995,Negative,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,يستحق ان بوكنق لانه سيء شي يوجد خدمة افطار صبا...
99996,Negative,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,كتاب ضعيف جدا ولم استمتع به كل قصه سرد لحاله ا...
99997,Negative,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,مملة جدا محمد حسن علوان فنان بالكلمات، والوصف ...


In [225]:
# encode label
label_mapping = {'Negative': -1, 'Positive': 1}
df['label'] = df['label'].str.strip()
df['label'] = df['label'].map(label_mapping)

df

,label,text,cleanedtext
0,1,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...
1,1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,احد اسباب نجاح الامارات ان كل شخص الدولة يعشق ...
2,1,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,هادفة وقوية تنقلك صخب شوارع القاهرة الي هدوء ج...
3,1,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,خلصنا مبدءيا اللي مستني ابهار زي الفيل الازرق ...
4,1,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,ياسات جلوريا جزء يتجزا دبي فندق متكامل الخدمات...
...,...,...,...
99994,-1,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,معرفش ليه كنت عاوزة اكملها وهي مش عاجباني البد...
99995,-1,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,يستحق ان بوكنق لانه سيء شي يوجد خدمة افطار صبا...
99996,-1,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,كتاب ضعيف جدا ولم استمتع به كل قصه سرد لحاله ا...
99997,-1,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,مملة جدا محمد حسن علوان فنان بالكلمات، والوصف ...


In [226]:
#TF-IDF vector
vectorizer = TfidfVectorizer()
x_tfidf = vectorizer.fit_transform(df['cleanedtext'])
y = df['label']

In [227]:
df

,label,text,cleanedtext
0,1,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...
1,1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,احد اسباب نجاح الامارات ان كل شخص الدولة يعشق ...
2,1,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,هادفة وقوية تنقلك صخب شوارع القاهرة الي هدوء ج...
3,1,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,خلصنا مبدءيا اللي مستني ابهار زي الفيل الازرق ...
4,1,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,ياسات جلوريا جزء يتجزا دبي فندق متكامل الخدمات...
...,...,...,...
99994,-1,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,معرفش ليه كنت عاوزة اكملها وهي مش عاجباني البد...
99995,-1,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,يستحق ان بوكنق لانه سيء شي يوجد خدمة افطار صبا...
99996,-1,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,كتاب ضعيف جدا ولم استمتع به كل قصه سرد لحاله ا...
99997,-1,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,مملة جدا محمد حسن علوان فنان بالكلمات، والوصف ...


In [228]:
print(x_tfidf)

  (0, 166761)	0.20741039236169664
  (0, 174685)	0.28389393697694404
  (0, 45423)	0.24635810832319477
  (0, 190391)	0.33666362510722125
  (0, 183799)	0.5839989197679492
  (0, 186617)	0.5340871332829625
  (0, 41480)	0.27576555885867055
  (1, 3463)	0.09402514109268716
  (1, 6604)	0.13605776593321817
  (1, 171166)	0.15057860557895755
  (1, 16428)	0.27885634901266065
  (1, 48166)	0.048906889535388375
  (1, 130786)	0.11476420561980656
  (1, 106403)	0.10495585526860568
  (1, 25204)	0.13321446923383456
  (1, 228021)	0.17297607994046937
  (1, 77322)	0.21825452520787408
  (1, 171567)	0.11375217130334649
  (1, 171445)	0.1585192698942597
  (1, 213433)	0.19386671094687122
  (1, 122228)	0.13664551200127414
  (1, 172555)	0.15937423461503425
  (1, 139117)	0.19621996708332773
  (1, 80190)	0.21216789237655967
  (1, 56434)	0.2414984123507325
  :	:
  (66664, 158546)	0.15383367144284255
  (66664, 47968)	0.17371444171246367
  (66664, 36607)	0.22849253263726105
  (66664, 95311)	0.22461289872811505
  (66664, 

# MODEL TRAINING

In [229]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf, y, test_size=0.1, random_state=42)

# Train classifier
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

# Predict and evaluate
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8572071396430179


# ARABERT TEST